API safe request rate is limit 1 request per second.

In [1]:
!pip install requests
!pip install diskcache

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [26]:
import requests
import base64
import json
import os
from datetime import datetime
import pytz
import diskcache

In [28]:
start_date = datetime.fromisoformat('1970-01-01T11:11:11+00:00')
last_update_date = None

In [3]:
project_dir = os.path.abspath('')

API = {
    "time_entries" : "https://api.track.toggl.com/api/v9/me/time_entries",
    "get_current_time_entry": "https://api.track.toggl.com/api/v9/me/time_entries/current", 
    "me_info" : "https://api.track.toggl.com/api/v9/me",
    "projects_me" : "https://api.track.toggl.com/api/v9/me/projects",
    "projects_ws" : "https://api.track.toggl.com/api/v9/workspaces/4143224/projects",
    "workspaces" : "https://api.track.toggl.com/api/v9/workspaces",
}

with open(f"{project_dir}/api_key.txt") as f:
    API_token = f"{f.read()}:api_token"
API_token_base64 = base64.b64encode(API_token.encode('ascii')).decode('ascii')

In [4]:
project_id_names = diskcache.Cache(f"{project_dir}/project_id_names.cache")

In [45]:
def fetch(url, headers={}):
    r = requests.get(url, headers={"Content-Type": "application/json", 
                                    "Host": "api.track.toggl.com",
                                    "authorization": f"Basic {API_token_base64}", 
                                    **headers})
    return json.loads(r.content.decode('utf-8'))

In [46]:
def fetch_project(project_id):
    return fetch(f"{API['projects_ws']}/{project_id}")

In [47]:
def filter_dict(d, keys):
    return list(map(lambda x: {k: v for k, v in x.items() if k in keys}, d))

In [48]:
def update_current_time_entry_duration(time_entry):
    time_entry["duration"] = (datetime.utcnow().replace(tzinfo=pytz.utc) - \
                              datetime.fromisoformat(time_entry["start"])).seconds
    return time_entry

In [49]:
l = {'c': 'x'}
{"a" : 'b', **l}

{'a': 'b', 'c': 'x'}

In [54]:
def get_time_entries(start_date=datetime.utcnow().replace(tzinfo=pytz.utc)):
    time_entries = [update_current_time_entry_duration(fetch(API["get_current_time_entry"]))]
    time_entries.extend(fetch(API["time_entries"], ))
    
    for time_entry in time_entries:
        project_id = time_entry['project_id']
        if project_id not in project_id_names:
            project_id_names[project_id] = fetch_project(project_id)
        time_entry['project_name'] = project_id_names[project_id]['name']
        
    return time_entries

In [55]:
get_time_entries()

ConnectionError: HTTPSConnectionPool(host='api.track.toggl.com', port=443): Max retries exceeded with url: /api/v9/me/time_entries/current (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fae60907dc0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))